# Model Training - Indonesian License Plate Detection

**Purpose:** Train YOLOv8 model directly on clean Roboflow dataset with optimal baseline parameters.

This notebook implements the optimized training approach from CLAUDE.md.

## 1. Import Libraries and Setup

In [1]:
import os
import sys
from pathlib import Path
import yaml
import json
import torch
from datetime import datetime
import shutil

# Import YOLOv8
from ultralytics import YOLO

print("📦 Libraries imported successfully")
print(f"Working directory: {os.getcwd()}")
print(f"PyTorch version: {torch.__version__}")

# Check GPU availability
if torch.cuda.is_available():
    print(f"✅ CUDA available")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU Memory: {gpu_memory:.1f} GB")
else:
    print("⚠️  CUDA not available - will use CPU (slower)")

📦 Libraries imported successfully
Working directory: c:\Users\Rafael Jonathan\Desktop\license-plate-training
PyTorch version: 2.7.1+cpu
⚠️  CUDA not available - will use CPU (slower)


## 2. Setup Dataset Configuration

In [2]:
# Define paths (portable structure with updated root)
BASE_DIR = Path("..")
DATASET_PATH = BASE_DIR / "dataset" / "plat-kendaraan"
MODELS_DIR = BASE_DIR / "models"
EXPERIMENTS_DIR = MODELS_DIR / "experiments"
FINAL_MODELS_DIR = MODELS_DIR / "final"
RESULTS_DIR = BASE_DIR / "results"

# Create directories
for directory in [EXPERIMENTS_DIR, FINAL_MODELS_DIR, RESULTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"🗂️  Dataset location: {DATASET_PATH}")
print(f"📁 Models directory: {MODELS_DIR}")

# Check if dataset exists
if DATASET_PATH.exists():
    data_yaml = DATASET_PATH / "data.yaml"
    if data_yaml.exists():
        print(f"✅ Dataset found with configuration: {data_yaml}")
        
        # Load and display config
        with open(data_yaml, 'r') as f:
            dataset_config = yaml.safe_load(f)
        
        print("\n📋 Dataset Configuration:")
        for key, value in dataset_config.items():
            print(f"  {key}: {value}")
        
        dataset_ready = True
    else:
        print(f"❌ data.yaml not found at: {data_yaml}")
        dataset_ready = False
else:
    print(f"❌ Dataset not found at: {DATASET_PATH}")
    print("Please run notebook 01 to download the dataset first.")
    dataset_ready = False

# Verify dataset structure
if dataset_ready:
    print("\n🔍 Dataset Structure:")
    for split in ['train', 'valid', 'test']:
        split_path = DATASET_PATH / split
        if split_path.exists():
            images = len(list((split_path / 'images').glob('*.jpg')))
            labels = len(list((split_path / 'labels').glob('*.txt')))
            print(f"  {split:>5}: {images:>5} images, {labels:>5} labels")
        else:
            print(f"  {split:>5}: ❌ Not found")

🗂️  Dataset location: ..\dataset\plat-kendaraan
📁 Models directory: ..\models
✅ Dataset found with configuration: ..\dataset\plat-kendaraan\data.yaml

📋 Dataset Configuration:
  names: ['License_Plate']
  nc: 1
  roboflow: {'license': 'MIT', 'project': 'vehicle-and-license-plate', 'url': 'https://universe.roboflow.com/plat-kendaraan/vehicle-and-license-plate/dataset/1', 'version': 1, 'workspace': 'plat-kendaraan'}
  test: ../test/images
  train: ../train/images
  val: ../valid/images

🔍 Dataset Structure:
  train:  5100 images,  5100 labels
  valid:   432 images,   432 labels
   test:    21 images,    21 labels


## 3. Configure Optimal Training Parameters

In [3]:
# Create experiment name
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
experiment_name = f"yolov8n_baseline_{timestamp}"

# Optimal baseline configuration (from CLAUDE.md)
training_config = {
    # Core parameters
    "data": str(DATASET_PATH / "data.yaml"),
    "epochs": 100,
    "patience": 15,      # Early stopping
    "batch": 16,         # Good balance for 8GB+ GPU  
    "imgsz": 640,        # Standard YOLO size
    "optimizer": "AdamW",
    "lr0": 0.001,        # Conservative learning rate
    "val": True,
    "save": True,
    "plots": True,
    
    # Output configuration
    "project": str(EXPERIMENTS_DIR),
    "name": experiment_name,
    "exist_ok": True,
    
    # Performance settings
    "device": 0 if torch.cuda.is_available() else "cpu",
    "workers": 4 if torch.cuda.is_available() else 2,
    "cache": False,      # Save memory
    "verbose": True
}

# Adjust batch size based on GPU memory
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    if gpu_memory < 8:
        training_config["batch"] = 8
        print(f"⚠️  Adjusted batch size to 8 for {gpu_memory:.1f} GB GPU")
    else:
        print(f"✅ Using batch size 16 with {gpu_memory:.1f} GB GPU")
else:
    training_config["batch"] = 4
    training_config["workers"] = 2
    print("⚠️  CPU mode: batch size 4, workers 2")

print("\n🎯 Training Configuration:")
print("=" * 30)
key_params = ["epochs", "batch", "imgsz", "optimizer", "lr0", "patience", "device"]
for key in key_params:
    print(f"{key:>10}: {training_config[key]}")

print(f"\n🚀 Experiment: {experiment_name}")
print(f"📊 Results will be saved to: {EXPERIMENTS_DIR / experiment_name}")

⚠️  CPU mode: batch size 4, workers 2

🎯 Training Configuration:
    epochs: 100
     batch: 4
     imgsz: 640
 optimizer: AdamW
       lr0: 0.001
  patience: 15
    device: cpu

🚀 Experiment: yolov8n_baseline_20250806_212145
📊 Results will be saved to: ..\models\experiments\yolov8n_baseline_20250806_212145


## 4. Load YOLOv8 Model

In [4]:
if dataset_ready:
    print("🤖 Loading YOLOv8n model...")
    
    try:
        # Load pre-trained YOLOv8 nano model
        model = YOLO('yolov8n.pt')
        print("✅ YOLOv8n model loaded successfully")
        
        # Display model info
        print("\n📋 Model Information:")
        model.info(verbose=False)
        
        model_ready = True
        
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        model_ready = False
else:
    print("⚠️  Skipping model loading - dataset not ready")
    model_ready = False

🤖 Loading YOLOv8n model...
✅ YOLOv8n model loaded successfully

📋 Model Information:


## 5. Start Training

In [ ]:
if dataset_ready and model_ready:
    print("🚀 Starting YOLOv8 training...")
    print(f"Target: mAP@0.5 > 0.85")
    print(f"Training will stop early if no improvement for {training_config['patience']} epochs\n")
    
    try:
        # Start training
        results = model.train(**training_config)
        
        print("\n🎉 Training completed successfully!")
        print(f"📊 Results saved to: {EXPERIMENTS_DIR / experiment_name}")
        
        training_success = True
        
    except KeyboardInterrupt:
        print("\n⏹️  Training interrupted by user")
        training_success = False
        
    except Exception as e:
        print(f"\n❌ Training failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("  • Reduce batch size if out of memory")
        print("  • Check dataset paths in data.yaml")
        print("  • Verify GPU/CUDA installation")
        training_success = False
        
else:
    print("❌ Cannot start training:")
    if not dataset_ready:
        print("  • Dataset not ready - run cells above")
    if not model_ready:
        print("  • Model not loaded - check previous cell")
    training_success = False

🚀 Starting YOLOv8 training...
Target: mAP@0.5 > 0.85
Training will stop early if no improvement for 15 epochs

New https://pypi.org/project/ultralytics/8.3.174 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.173  Python-3.13.2 torch-2.7.1+cpu CPU (Intel Core(TM) Ultra 7 155H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=..\dataset\plat-kendaraan\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, mod

train: Scanning C:\Users\Rafael Jonathan\Desktop\dataset\plat-kendaraan\train\labels... 5100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5100/5100 [00:23<00:00, 219.48it/s]


train: New cache created: C:\Users\Rafael Jonathan\Desktop\dataset\plat-kendaraan\train\labels.cache
WARNING Box and segment counts should be equal, but got len(segments) = 1634, len(boxes) = 5364. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.20.1 ms, read: 5.20.8 MB/s, size: 72.6 KB)


c:\Users\Rafael Jonathan\Desktop\license-plate-training\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\Rafael Jonathan\Desktop\dataset\plat-kendaraan\valid\labels... 432 images, 0 backgrounds, 0 corrupt: 100%|██████████| 432/432 [00:03<00:00, 143.88it/s]

val: New cache created: C:\Users\Rafael Jonathan\Desktop\dataset\plat-kendaraan\valid\labels.cache
WARNING Box and segment counts should be equal, but got len(segments) = 143, len(boxes) = 463. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



c:\Users\Rafael Jonathan\Desktop\license-plate-training\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to ..\models\experiments\yolov8n_baseline_20250806_212145\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to ..\models\experiments\yolov8n_baseline_20250806_212145
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      1.309       1.18      1.177          4        640:  74%|███████▍  | 947/1275 [19:06<09:06,  1.67s/it]

## 6. Training Results Analysis

In [ ]:
if 'training_success' in locals() and training_success:
    print("📊 Analyzing training results...")
    
    # Find experiment directory
    exp_dir = EXPERIMENTS_DIR / experiment_name
    
    if exp_dir.exists():
        print(f"✅ Experiment directory: {exp_dir}")
        
        # Check for results files
        results_csv = exp_dir / "results.csv"
        if results_csv.exists():
            import pandas as pd
            
            # Load training metrics
            df = pd.read_csv(results_csv)
            print(f"\n📈 Training completed in {len(df)} epochs")
            
            # Extract key metrics from final epoch
            final_metrics = df.iloc[-1]
            
            print("\n🎯 Final Performance:")
            print("=" * 25)
            
            # Key metrics to display
            metric_names = {
                'metrics/mAP50(B)': 'mAP@0.5',
                'metrics/mAP50-95(B)': 'mAP@0.5:0.95', 
                'metrics/precision(B)': 'Precision',
                'metrics/recall(B)': 'Recall',
                'train/box_loss': 'Box Loss',
                'val/box_loss': 'Val Box Loss'
            }
            
            for col, name in metric_names.items():
                if col in df.columns:
                    value = final_metrics[col]
                    if 'loss' in col.lower():
                        print(f"{name:>12}: {value:.4f}")
                    else:
                        print(f"{name:>12}: {value:.3f}")
            
            # Check if target mAP@0.5 > 0.85 was achieved
            map50_col = 'metrics/mAP50(B)'
            if map50_col in df.columns:
                final_map50 = final_metrics[map50_col]
                target_map50 = 0.85
                
                print(f"\n🎯 Performance vs Target:")
                print(f"Final mAP@0.5: {final_map50:.3f}")
                print(f"Target mAP@0.5: {target_map50:.3f}")
                
                if final_map50 >= target_map50:
                    print("✅ TARGET ACHIEVED! Model ready for production.")
                else:
                    gap = target_map50 - final_map50
                    print(f"⚠️  Target not reached (gap: {gap:.3f})")
                    print("💡 Consider: upgrade to yolov8s, increase epochs, or tune hyperparameters")
        else:
            print("⚠️  results.csv not found")
            
        # List generated files
        print(f"\n📁 Generated files in {exp_dir}:")
        for item in exp_dir.rglob('*'):
            if item.is_file():
                rel_path = item.relative_to(exp_dir)
                print(f"  📄 {rel_path}")
                
    else:
        print(f"❌ Experiment directory not found: {exp_dir}")
        
else:
    print("⚠️  No training results to analyze")

## 7. Export Best Model for Production

In [ ]:
if 'training_success' in locals() and training_success:
    print("📦 Exporting best model for production...")
    
    exp_dir = EXPERIMENTS_DIR / experiment_name
    weights_dir = exp_dir / "weights"
    best_weights = weights_dir / "best.pt"
    
    if best_weights.exists():
        # Copy best model to final models directory
        final_model_name = f"yolov8n_indonesian_plates_{timestamp}.pt"
        final_model_path = FINAL_MODELS_DIR / final_model_name
        
        shutil.copy2(best_weights, final_model_path)
        print(f"✅ Model saved as: {final_model_path}")
        
        # Create best_model.pt link (as specified in CLAUDE.md)
        best_model_link = FINAL_MODELS_DIR / "best_model.pt"
        if best_model_link.exists():
            best_model_link.unlink()
        
        shutil.copy2(final_model_path, best_model_link)
        print(f"✅ Created production link: {best_model_link}")
        
        # Check model size (target < 50MB from CLAUDE.md)
        model_size_mb = final_model_path.stat().st_size / (1024 * 1024)
        print(f"📏 Model size: {model_size_mb:.1f} MB")
        
        if model_size_mb < 50:
            print("✅ Model size meets deployment requirement (< 50MB)")
        else:
            print(f"⚠️  Model exceeds 50MB target for deployment")
        
        # Test model loading
        print("\n🧪 Testing model loading...")
        try:
            test_model = YOLO(str(final_model_path))
            print("✅ Model loads successfully")
            
            # Model info
            print(f"Model classes: {test_model.names}")
            print(f"Model device: {test_model.device}")
            
        except Exception as e:
            print(f"❌ Model loading failed: {e}")
        
        print(f"\n🚀 Model ready for production integration!")
        print(f"Next: Run notebook 05_evaluation_export.ipynb for detailed evaluation")
        
    else:
        print(f"❌ Best weights not found at: {best_weights}")
        print("Check if training completed successfully")
        
else:
    print("⚠️  No trained model to export")

## Summary

### ✅ Training Configuration Used:
- **Model**: YOLOv8n (optimized for speed/size balance)
- **Epochs**: 100 with early stopping (patience=15)
- **Batch Size**: 16 (GPU) / 8 (low memory) / 4 (CPU)
- **Image Size**: 640×640 (standard YOLO)
- **Optimizer**: AdamW with lr=0.001

### 🎯 Performance Targets (CLAUDE.md):
- **Accuracy**: mAP@0.5 > 0.85 ✅/⚠️
- **Speed**: < 100ms per image (GPU inference)
- **Model Size**: < 50MB for deployment
- **Confidence**: >= 0.3 threshold

### 💡 If Target Not Met - Tuning Options:
1. **Model Upgrade**: yolov8n → yolov8s → yolov8m
2. **Batch Size**: 8/16/32 (GPU memory dependent)
3. **Image Size**: 640 → 832 → 1024 (accuracy vs speed)
4. **Learning Rate**: 0.0005 → 0.002 (fine-tuning)

### 🚀 Next Steps:
1. **Check training results above**
2. **Run notebook 05_evaluation_export.ipynb for detailed evaluation**
3. **Test model on sample images**
4. **Integrate with production system**